## weird stuff: mutating immutables

In [ ]:
my_tuple = (1, 2, [30, 40])

what happens if I try to change the mutable list inside the immutable tuple through augmented assignments (`+=`)?
- my_tuple becomes `(1, 2, [30, 40, 50])`
- An exception is raised, since I'm changing the content of an immutable
- both
- neither
- something else


In [ ]:
my_tuple[-1] += [50]

In [ ]:
my_tuple

what happens if I try to change the mutable list insie the immutable tuple through a list method (`append`)

In [ ]:
my_tuple[-1].append(60)

In [ ]:
my_tuple

In [ ]:
# for completeness sake: this is due to `__iadd__` in List
from collections import UserList
class MyList(UserList):
    def __iadd__(self, other):
        return NotImplemented

In [ ]:
my_list = MyList([30, 40])

In [ ]:
my_tuple = (1, 2, my_list)

In [ ]:
my_tuple[-1] += [50]

In [ ]:
my_tuple

### Summary:
- Putting mutables inside immutables is not a good idea
- augmented assignment (+=) is /not/ an atomic operations, and so you get the 'both' answer above...

## Dictionaries and Sets: How do they actually work?

### set and dict lookups are blazing fast

In [ ]:
import random, string

In [ ]:
haystack = [random.random() for _ in range(1000000)]
needles = random.sample(haystack, 1000)

haystack_set = set(haystack)
needles_set = set(needles)

In [ ]:
# find 1000 needles in a haystack of 1M entries

In [ ]:
%timeit [needle for needle in needles if needle in haystack];

In [ ]:
%timeit haystack_set.intersection(needles_set)

In [ ]:
# the first element in the list is much faster than the last...

In [ ]:
%timeit haystack[0] in haystack

In [ ]:
%timeit haystack[-1] in haystack

In [ ]:
%timeit haystack[0] in haystack_set

In [ ]:
%timeit haystack[-1] in haystack_set

### how is this achieved?
- https://github.com/python/cpython/blob/main/Objects/dictobject.c

In [ ]:
import string

In [ ]:
# assume we want to re-create the functionality python offers for a dict like this here:
dict = {'first_key': 13, 'second_key': 'some_value'}

In [ ]:
# the list way of doing something that looks like a dict...

class ListDict:
    def __init__(self, **kwargs):
        self._data = []
        for k, v in kwargs.items():
            self[k] = v

    # adding elements to our 'dict'
    def __setitem__(self, key, value):
        self._data.append((key, value))

    # getting elements from our 'dict'
    def __getitem__(self, key):
        for elem in self._data:  # iterate over all elements in the list!
            if elem[0] == key:
                return elem[1]

    def __contains__(self, key):
        for elem in self._data:  # iterate over all elements in the list!
            if elem[0] == key:
                return True
        return False

In [ ]:
ld = ListDict(first_key=13, second_key='some_value')

In [ ]:
ld['first_key']

In [ ]:
'second_key' in ld

In [ ]:
'some_key' in ld

In [ ]:
# the hash-table value of building a dict
# what if, instead of searching through the list we could just /calculate/ which element is the correct one?!


class HashDict:
    def __init__(self, size=3, **kwargs):
        self.bit_selector = 2**size - 1
        self._data = [None]*2**size  # extra memory allocation here!
        for k, v in kwargs.items():
            self[k] = v

    def _get_index(self, key):
        return hash(key) & self.bit_selector

    def __getitem__(self, key):
        index = self._get_index(key)
        element = self._data[index]  # whoa, no loop!
        if element is None:
            raise KeyError
        return element[1]

    def __setitem__(self, key, value):
        index = self._get_index(key)
        self._data[index] = (key, value)

    def __contains__(self, key):
        index = self._get_index(key)
        return not self._data[index] is None
            

In [ ]:
hd = HashDict(first_key=13, second_key='some_value')

In [ ]:
hd['first_key']

In [ ]:
'second_key' in hd

In [ ]:
'some_key' in hd

In [ ]:
# what's a problem with our implementation?

In [ ]:
'foo' in hd

In [ ]:
# let's make it larger
hd = HashDict(size=4, first_key=13, second_key='some_value')
'foo' in hd

In [ ]:
# that was a hash collision: same (lower bits) hash for two different objects

In [ ]:
# let's try to do better

class CollisionSafeHashDict:
    def __init__(self, size=3, **kwargs):
        self.bit_selector = 2**size - 1
        self._data = [None]*2**size  # extra memory allocation here!
        for k, v in kwargs.items():
            self[k] = v

    def _get_index(self, key, shift=0):
        if shift >= 8:
            raise RuntimeError('our implementation is too simple to deal with resizing the dict')
        selected_bits = hash(key) & (self.bit_selector << shift)  # the real logic here is smarter...
        return selected_bits >> shift  # bring it into index range again

    def __getitem__(self, key):
        for shift in range(16):
            index = self._get_index(key, shift)
            element = self._data[index]  # whoa, no loop!
            if element is None:
                raise KeyError
            elif element[0] == key:
                return element[1]

    def __setitem__(self, key, value):
        for shift in range(16):
            index = self._get_index(key, shift)
            if self._data[index] is None:
                self._data[index] = (key, value)
                break
            elif self._data[index][0] == key:
                self._data[index] = (key, value)
                break
            

    def __contains__(self, key):
        for shift in range(16):
            index = self._get_index(key, shift)
            if self._data[index] is None:
                return False
            elif self._data[index][0] == key:
                return True
        return False

In [ ]:
chd = CollisionSafeHashDict(first_key=13, second_key='some_value')  # using the old size here

In [ ]:
'foo' in chd

In [ ]:
'first_key' in chd

In [ ]:
'second_key' in chd

In [ ]:
for i in string.ascii_lowercase:
    chd[i] = 'value'

In [ ]:
chd._data

### Summary
- classic time-space-tradeoff
- significant memory overhead for dictionaries
- all keys (and all set elements) must be hashable
- element retrieval and membership tests are very fast (O(1) effectively)
- dicts have some dedicated logic to conserve order of entries
- for sets the order is not defined, and any insert may change the existing order of all elements
- inserts can (ocassionally) trigger a re-sizing of the underlying data structure

## Debugging

### Python Debugger (`pdb`) Commands

#### Starting the Debugger
- known breakpoint in advance: `breakpoint()`
- break into debugger on unhandled exceptions: `python -m pdb <my_script.py>`

#### Navigation
- **`n` or `next`**: Execute the current line and stop at the first possible occasion (step over).

- **`s` or `step`**: Execute and stop at the first possible occasion (step into).

- **`c` or `continue`**: Continue execution until the next breakpoint is encountered.

#### Breakpoints
- **`b` or `break`**: Set a breakpoint at the current line.

- **`b <line_number>`**: Set a breakpoint at a specific line.

- **`clear <breakpoint_number>`**: Clear a breakpoint.

#### Information
- **`l` or `list`**: Show the current source code around the current line.

- **`p <variable>` or `print <variable>`**: Print the value of a variable.

- **`q` or `quit`**: Quit the debugger and abort the program.

#### Stack and Context
- **`bt` or `where`**: Print a stack trace with the current line number.

- **`u` or `up`**: Move the current frame up the stack trace.

- **`d` or `down`**: Move the current frame down the stack trace.

#### Miscellaneous
- **`h` or `help`**: Display a list of available commands or help for a specific command.

- **`!`**: Execute a Python statement in the context of the current stack frame.


## typing and type hints
- type annotations have special syntax
- but are **entirely optional**
- they can be used by static types checkers, documentation, IDEs or just for better understanding by the reader
- but they *do not* influence the interpreter in any way
- in particular there is no runtime effect: nothing gets faster (or slower) because you provided type annotations
- and nothing is enforced by the interpreter either

In [ ]:
# motivating example

In [ ]:
def some_function(some_argument):
    some_argument.

In [ ]:
def some_function(some_argument: str):
    some_argument.

### types for variables

In [ ]:
x: int = 5  # syntax to specify x should be an integer

In [ ]:
x = 'foo'  # the interpreter doesn't care at all 

In [ ]:
# you can use all the builtin types as type hint
s: str = 'foo'
b: bool = True
# and so on

In [ ]:
# simple list type
l: list = []

In [ ]:
# generic types
list_of_strings: list[str] = []

In [ ]:
def foo(l: list[str]) -> None:
    l[1].

In [ ]:
# but, lists are expected to be homogeneous; you can not specify different types per element
def foo(l: list[str, int]) -> None:
    l[1].

In [ ]:
# same for tuples or sets
t: tuple = ()
t: tuple[int] = ()

In [ ]:
# but tuples are often inhomogeneous, so the following /is/ officially allowed:
# unfortunately, both examples are only theoretical for the type hint system in this notebook...
# see vscode, which does better here...
def foo(t: tuple[str, int]) -> None:
    t[1].

In [ ]:
# for a homogeneous tuple of any length use Ellipsis 
t: tuple[int, ...] = (1, 2, 3, 4, 5)

In [ ]:
# sets are simple again, they are expected to be homogeneous
s: set = set()
s: set[float] = set()

In [ ]:
# dictionaries follow a fairly natural syntax, too
d: dict[str, int] = {'key': 1}

### abstract base classes as types

In [ ]:
from collections.abc import Iterable, Sequence, Mapping
from collections import UserList, UserDict


class MyIterable(Iterable):
    def __iter__(self):
        return (x for x in [1, 2, 3])

class MyList(UserList):
    pass

class MyDict(UserDict):
    pass

# if you just expect something to be iterable (implements `__iter__`), no matter what exactly it is
iterable: Iterable = MyIterable()
# if you just expect something to implement the sequence protocal (`__getitem__`, `__len__`), no matter what exactly it is
sequence: Sequence = MySequence([1, 2, 3])
# if you just expect something to implement the mapping protocal (`__getitem__`, `__len__`, `__iter__`), no matter what exactly it is
d: Mapping = MyDict({'foo': 'bar'})



### types for functions

In [ ]:
def my_function(first_argument: str, second_argument: int = 3) -> bool:
    print(f'{first_argument=}, {second_argument=}')
    return True

In [ ]:
my_function('foo', 0)

In [ ]:
# the default value keeps working of course
my_function('foo')

In [ ]:
# and - as before - nothing is enforced!
my_function([1, 2, 3])

### type unions

In [ ]:
from typing import Union

In [ ]:
x: Union[str, float] = 3.14

In [ ]:
x: str | float = 3.14  # use `\` instead of union (since python 3.10)

In [ ]:
# you can also make it explicit that a value can be None
def some_function(optional_integer: int | None) -> None:
    if optional_integer is None:
        print('I got nothin')

In [ ]:
some_function(None)

In [ ]:
some_function(5)

In [ ]:
# this is independent of a default value you might also give it
# here we have it with default
def some_function(optional_integer: int | None = None) -> None:
    if optional_integer is None:
        print('I got nothin')

### using classes as types

In [ ]:
class Car:
    def __init__(self, make, model):
        self.make = make
        self.model = model

    def describe(self):
        print(f'This is a shiny new {self.make} {self.model}')

def car_consumer(car: Car) -> None:
    car.

In [ ]:
def car_factory(model: str) -> Car:
    model_map = {
        'Focus': 'Ford',
        'TT': 'Audi'
    }
    return Car(model_map[model], model)

In [ ]:
car_factory('Focus')

In [ ]:
car_factory('Focus').

In [ ]:
# what if you actually want the class as argument?
class User: pass
class TechnicalUser(User): pass
class AdminUser(TechnicalUser): pass

# this actually expects an **instance** of the class. But the function really wants the class!
def user_factory(user_class: User) -> User:
    pass

# this does actually expect the class, and returns an instance!
# thanks to inheritance subclasses are also subtypes, and hence also allowed here
def user_factory(user_class: type[User]) -> User:
    pass

### Special Types
**Type Aliases**

In [ ]:
# this is unreadable already, and just imagine you actually need this tye in multiple places
# (which is common)
def send_message(message: str, server: tuple[tuple[str, int], dict[str, str]]) -> None:
    pass

In [ ]:
# define an alias simply using the assignment operator
ConnectionOptions = dict[str, str]
Address = tuple[str, int]
Server = tuple[Address, ConnectionOptions]
def send_message(message: str, server: Server) -> None:
    pass

In [ ]:
# you might want to annotate those aliases, to make it clear they are a type
from typing import TypeAlias
ConnectionOptions: TypeAlias = dict[str, str]

**custom (semantic) types**

In [ ]:
from typing import NewType

In [ ]:
# the only thing indicating user_id is not just any odd int, but a 'user_id' is the name
# for the return value of the function that connection is ever more tenuous
def get_user_name(user_id: int) -> str:
    pass

In [ ]:
# always derived from some 'base type'
UserIDType = NewType('UserIDType', int)
UserNameType = NewType('UserNameType', str)

def get_user_name(user_id: UserIDType) -> UserNameType:
    return UserNameType('Zaphod Beeblebrox')

user_id = UserIDType(42)
get_user_name(user_id)


**types for callables**

In [ ]:
# example from week 2
def formatter(s):
    return f'Hey, look at that! ==> "{s}"'

def some_function(x, y, formatting_function):
    my_text = f'The sum of {x} and {y} is {x + y}'
    return formatting_function(my_text)

some_function(3, 5, formatter)

In [ ]:
# annotating the formatter is easy
def formatter(s: str) -> str:
    return f'Hey, look at that! ==> "{s}"'

In [ ]:
# but what about `some_function`, which has as function as an argument?
from typing import Callable

def some_function(x: int, y: int, formatting_function: Callable[[str], str]) -> str:
    my_text = f'The sum of {x} and {y} is {x + y}'
    return formatting_function(my_text)

`Callable`: the first 'argument' is the list of parameters of the function, the second is the return type!

## iterables, iterators, and generators and the like

### iterables, iterators and their relationship
- iterable: 'An object capable of returning its members one at a time.'
  - so for example list, tuple, string, ...
  - when calling the built-in `iter` on an `iterable`, what you get is an `iterator`
- iterator: 'An object representing a stream of data'
  - used internally in for loops, comprehensions, lopping over file contents, ...
  - each iterator can only be traversed once
  - there is **no need** to have the data in memory before or after an element is requested

#### Basics

In [ ]:
my_list = [1, 2, 3]
for element in my_list:
    print(element)

In [ ]:
iterator = iter(my_list)
iterator

In [ ]:
# calling `next` on the iterator returns the next value
next(iterator)

In [ ]:
next(iterator)

In [ ]:
next(iterator)

In [ ]:
# if the iterator is `exhausted` (no more elements to return) it raises `StopIteration`
next(iterator)

In [ ]:
# re-building the above for loop by hand
my_list = [1, 2, 3]
iterator = iter(my_list)
while True:
    try:
        print(next(iterator))
    except StopIteration:
        print('Iterator is exhausted')
        break

In [ ]:
# can only use iterator once, it remains exhausted (and I should have deleted it in the exception handler above)
next(iterator)

In [ ]:
# another way of showing exaustion of the iterator
iterator = iter(my_list)
for elem in iterator:
    print(elem)
for elem in iterator:
    print(elem)

#### Building your own

In [ ]:
import requests

In [ ]:
# Iterables implement `__iter__` and iterators implement `__iter__` and `__next__`
class TitanicPassengers:
    def __iter__(self):
        return TitanicIterator()

class TitanicIterator:
    def __init__(self):
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        response = requests.get(f'http://127.0.0.1:3333/passenger/{self.index}')
        if response.ok:
            self.index += 1
            return response.json()
        else:
            raise StopIteration

In [ ]:
for passenger in TitanicPassengers():
    print(passenger['Name'])

In [ ]:
passenger

In [ ]:
iter(TitanicPassengers())  # gives us our Iterator

In [ ]:
# waaaait a second, that's super lengthy, and we already saw an easier form, just implementing __getitem__
class OldfashionedTitanicPassengers:
    def __getitem__(self, index):
        response = requests.get(f'http://127.0.0.1:3333/passenger/{index}')
        if response.ok:
            return response.json()
        else:
            raise IndexError

In [ ]:
for passenger in OldfashionedTitanicPassengers():
    print(passenger['Name'])

In [ ]:
iter(OldfashionedTitanicPassengers())  # gives us a 'generic' iterator

**Summary**
- Iterables have to implement `__iter__`, returning an `iterator`
- `iterator`s have to implement `__iter__` and `__next__`, returning `self` and the next element respectively
- ...
- **except** if the iterable implements only `__getitem__` then `iter` will automatically construct the iterator for you
- but this is legacy behaviour, you /should/ use `__iter__`.
- but that's annoying, because we have to have a whole class for the `iterator` that doesn't do anything interesting

### generator functions to the rescue
- any function that has the `yield` keyword in it's body is called a `generator function`
- calling a generator function immediately returns a `generator`
- and `generator`s are pretty much the same as `iterator`s above
- `yield` is sort of like a `return` for the `generator` -- execution continues after the last `yield` if `next` is called on the `generator`
- any (explicit or implicit) `return` in the generator actually raises `StopIteration`

In [ ]:
def generate_123():
    print('start')
    yield 1
    print('in between')
    yield 2
    print('still in between')
    yield 3
    print('now we\'re done')

In [ ]:
generator = generate_123()
generator

In [ ]:
next(generator)

In [ ]:
next(generator)

In [ ]:
next(generator)

In [ ]:
next(generator)

In [ ]:
for element in generate_123():
    print(element)

In [ ]:
# so let's re-do our Titanic example using generator functions
class TitanicPassengerGenerator:
    def __iter__(self):
        index = 0  # since this is a generator function, the value is preserved between successive 'calls' (without explicit closure)
        while True:
            response = requests.get(f'http://127.0.0.1:3333/passenger/{index}')
            if response.ok:
                index += 1
                yield response.json()
            else:
                break

In [ ]:
for passenger in TitanicPassengerGenerator():
    print(passenger['Name'])

In [ ]:
iter(TitanicPassengerGenerator())  # what we get from calling `iter` is a `generator object`

### `yield from` as shortcut for nesting generators
- yield from gets an **iterator** from the thing to the right of it
- it then keeps yielding from that iterator until the iterator is exhausted
- during this time the outer function is effectively suspended
- after the iterator is exhausted execution continues in the outer function

In [ ]:
def my_generator():
    yield from 'ABC'
    yield from range(3)

In [ ]:
for element in my_generator():
    print(element)

In [ ]:
# a more interesting example (from the python cookbook, https://github.com/dabeaz/python-cookbook/blob/master/src/4/how_to_flatten_a_nested_sequence/example.py)
from collections.abc import Iterable

def flatten(items):
    for x in items:
        if isinstance(x, Iterable):
            yield from flatten(x)
        else:
            yield x

In [ ]:
items = [1, 2, [3, 4, [5, 6], 7], 8]
items

In [ ]:
flat_items = flatten(items)
flat_items

In [ ]:
list(flat_items)

### and generator expressions

In [ ]:
# we all remember list comprehensions
# list comprehensions realize the list in memory upon execution
out1 = [element*3 for element in generate_123()]

In [ ]:
out1

In [ ]:
# what looks like a 'tuple expression' is in reality a `generator expression`
out2 = (element*3 for element in generate_123())  # function was not actually called!?

In [ ]:
out2  # out2 is a 'generator object', just like what we got above for the titanic example

In [ ]:
for elem in out2:  # 
    print(elem)

In [ ]:
# also possible the other way round
my_iterator = (3*x for x in [1, 2, 3])
my_iterator

In [ ]:
# and we can also just return those objects in `__iter__`
class MyIterable:
    def __iter__(self):
        return (3*x for x in [1, 2, 3])

In [ ]:
for element in MyIterable():
    print(element)

In [ ]:
# itertools gives us all kinds of nifty built-in generator functionality
import itertools

In [ ]:
# now we understand why I always needed `list` in those examples
list(itertools.combinations('ABC', 2))

In [ ]:
abc_combinations = itertools.combinations('ABC', 2)
abc_combinations

In [ ]:
next(abc_combinations)

In [ ]:
next(abc_combinations)

In [ ]:
next(abc_combinations)

In [ ]:
next(abc_combinations)

### passing data into generators ('coroutines')
- Coroutines are a more generalized form of functions. Functions are entered at one point and exited at another point. Coroutines can be entered, exited, and resumed at many different points.

In [ ]:
# instead, the keyword here is nonlocal!
def make_averager_nonlocal():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [ ]:
my_averager_global = make_averager_nonlocal()
my_averager_global(10)
my_averager_global(11)
my_averager_global(12)
my_averager_global(13)

In [ ]:
def make_coro_averager():
    count = 0
    total = 0
    new_value = yield None
    while True:
        total += new_value
        count += 1
        new_value = yield total / count

In [ ]:
my_coro_averager = make_coro_averager()
next(my_coro_averager)  # priming the co-routine

In [ ]:
my_coro_averager.send(10)
my_coro_averager.send(11)
my_coro_averager.send(12)
my_coro_averager.send(13)

In [ ]:
# priming is necessary, can't send before priming

In [ ]:
my_coro_averager = make_coro_averager()
my_coro_averager.send(10)

In [ ]:
my_coro_averager = make_coro_averager()
my_coro_averager.send(None)  # can also prime by `send`ing `None`
my_coro_averager.send(10)
my_coro_averager.send(11)
my_coro_averager.send(12)
my_coro_averager.send(13)

In [ ]:
my_coro_averager = make_coro_averager()
my_coro_averager.send(None)  # prime the coroutine
try:
    my_coro_averager.send('foo')  # exceptions terminate the coroutine execution
except TypeError:
    print('There was an exception in the coroutine')

In [ ]:
# so I can't use it any further, trying to do so raises `StopIteration` (indicating the coroutine is exhausted
my_coro_averager.send(10)

In [ ]:
# calling `next` is the same as `send(None)`
my_coro_averager = make_coro_averager()
next(my_coro_averager)  # priming, advance to the first yield
next(my_coro_averager)  # continue from the first yield, with an implicit `None` assigned to the variable

### Summary
- to make an object **iterable** it should implement the `__iter__` method, and return an **iterator** from that method
- you get an iterator from an iterable using `iter(iterable)`
- you get elements from the iterator using `next(iterator)`
- there are different flavours of iterators:
  - explicit iterators, objects that implement `__next__` (and `__iter__`).
  - generator functions, functions that have `yield` somewhere in their body automatically return a `generator` (which is a a flavour of an iterator)
  - `generator expressions`
- Explicitly implementing your own iterator is almost never necessary
- generator functions suspend whenever they encounter `yield`, and pick up execution from that point whenever they are advanced again using `next`
- You can use generators as coroutines (allowing you to send data into them) using `<name> = yield <value>`
  - you have to prime these coroutines with an initial call to `next` (or `send(None)`)
  - after priming you feed data into them using `send` (and get intermediate results back from the `yield`)

<div class="alert alert-block alert-info">
<b>Reminder:</b> <br>
<a>
    the `itertools` is all about generators with lots of built-in functionality for those
</a>
</div>

<div class="alert alert-block alert-info">
<b>Outlook:</b> <br>
<a>
    understanding 'simple' co-routines is important for later understanding 'asynchronous' co-routines and the functionality of `async`/`await` and the `asyncio` package <br>
    unfortunately, there are many subtleties to sub-generators using `yield from` that go beyond the material discussed here
</a>
</div>